<H2 style="color:blue;font-size:20px;"align="right">فاطمه شاه حسینی</H2>
 
<H2 style="color:blue;font-size:20px;"align="right"> 810199440 </H2>
<H2 style="color:blue;font-size:20px;"align="right"> تمرین کامپیوتری شماره دو - الگوریتم ژنتیک </H2>
<H2 style="color:green;font-size:20px;"align="right"> هدف </H2>
<H2 style="color:black;font-size:20px;"align="right">پیدا کردن پاسخ مناسب با لحاظ کردن محدودیت های مساله به کمک الگوریتم ژنتیک</H2>
<H2 style="color:green;font-size:20px;"align="right"> توضیح کلی پروژه </H2>
<H2 style="color:black;font-size:20px;"align="right"> باید به کمک الگوریتم ژنتیک ضرایب مناسب برای هر سهم به منظور ارتفاع محدودیت بازده و ریسک مساله را به دست آوریم. ابتدا یک جمعیت اولیه به طور رندوم می سازیم. با استفاده از
ترکیب کروموزوم ها -کراس اور- و جهش ژن ها-میوتیشین-جمعیت جدیدی تولید می کنیم و نسل جدید را با کروموزوم  هایی که فیتنس بیشتری دارند، پر می کنیم . اینکار را ادامه
می دهیم تا جایی که کروموزوم تولید شده، محدودیت های مساله ما را ارضا کند</H2>
<H2 style="color:green;font-size:20px;"align="right"> تطبیق مفاهیم اولیه الگوریتم ژنتیک با مساله </H2>
<H2 style="color:black;font-size:20px;"align="right"> دراین مساله ، ضریب مربوط به هر سهام یک ژن است که مجموعه ضرایب کنار هم یک کروموزوم یا یک پاسخ برای مساله تشکیل می دهند.مجموعه ای از کروموزوم ها هم جعمیت و یک نسل را نشان می دهند. </H2>

In [54]:
import pandas as pd
import numpy as np
import random
import time
from copy import deepcopy
MIN_FITNESS = 10 - 0.6
max_len = 50

<H2 style="color:green;font-size:20px;"align="right">خواندن اطلاعات سهام ها از فایل </H2>

In [55]:
stock_data = pd.read_csv("./Test/sample.csv")
coeff_data = pd.read_csv("./Test/sample_coeffs.csv")
cf = coeff_data['coeffs']
nz =  np.where(cf != 0)[0]

In [57]:
a = stock_data.sort_values("return").head(10)
b = a['Unnamed: 0'].to_numpy() 

<H2 style="color:green;font-size:20px;"align="right"> تابع محاسبه فیتنس </H2>
<H2 style="color:black;font-size:20px;"align="right">از آنجا که شرط محدودیت روی بازده بیشتر سخت تر از ریسک است، مقدار تاثیر بازده در فیتنس را بالاتر می بریم. با توجه به فیتنس دوتا کروموزوم را با هم مقایسه می کنیم </H2>

In [59]:
def calc_fitness(chromosome, stocks = stock_data):
    chromosome = np.array(chromosome)
    not_zero_stocks_index = np.where(chromosome != 0)[0]
    total_return = 0
    total_risk = 0
    for i in not_zero_stocks_index:
        stock = stocks.loc[i]
        total_return += chromosome[i] * stock['return']
        total_risk += chromosome[i] * stock['risk']
    return (total_return*total_return / total_risk)

In [60]:
def calc_final_fitness(chromosome, stocks = stock_data):
    chromosome = np.array(chromosome)
    not_zero_stocks_index = np.where(chromosome != 0)[0]
    total = 0
    for i in not_zero_stocks_index:
        stock = stocks.loc[i]
        total += chromosome[i] * (stock['return'] - stock['risk'])
    return total

<H2 style="color:green;font-size:20px;"align="right"> ساخت جمعیت اولیه </H2>
<H2 style="color:black;font-size:20px;"align="right"> به طور رندوم ژن ها(ضرایب سهام ها) را تولید می کنیم و با کنار هم قرار دادن آنها، کروموزوم ها را می سازیم.  </H2>

In [61]:
def create_population(population_len = max_len, stocks_len = len(stock_data.index)):
    population = []
    for j in range(population_len):
        chromosome = []
        rand_coef = random.randint(1, 10)
        chromosome.append(rand_coef)
        for i in range(stocks_len - 1):
            if(random.randint(1, 10) < 2):
                rand_coef = random.randint(1, 10)
                chromosome.append(rand_coef)
            else:
                chromosome.append(0)
        gen_sum = sum(chromosome)
        chromosome = [g / gen_sum for g in chromosome]
        population.append(chromosome)
    return population    

<H2 style="color:green;font-size:20px;"align="right"> Crossover function </H2>
<H2 style="color:black;font-size:20px;"align="right"> این تابع دو کروموزم را از نقطه ای به طور رندوم قطع می کند و به طور ضربدری دوباره وصل می کند و دو فرزند تولید می کند  </H2>

In [62]:
def crossover(chr1, chr2):
    gen_sum1, gen_sum2 = 0, 0
    x = random.randint(1, len(chr1) - 1)
    new_chr1 = chr1[:x] + chr2[x:]
    gen_sum1 = sum(new_chr1)
    new_chr2 = chr2[:x] + chr1[x:]
    gen_sum2 = sum(new_chr2)
    new_chr1 = [g / gen_sum1 for g in new_chr1]
    new_chr2 = [g / gen_sum2 for g in new_chr2]
    return new_chr1, new_chr2

<H2 style="color:green;font-size:20px;"align="right"> Mutation function </H2>
<H2 style="color:black;font-size:20px;"align="right"> در این تابع چند جفت از ژن ها به طور رندوم انتخاب شده و با هم جابه جا می شوند. در صورتی که فیتنس کروموزوم جدید بیشتر از قبلی باشد، تغییر اعمال می شود وگرنه همان 
کروموزوم قبلی برگردانده می شود.</H2>

In [68]:
def mutate(chr):
    min_fitness = calc_fitness(chr)
    new_chr = deepcopy(chr)
    x, y = 0,0
    for i in range(5):
        x, y = random.randint(0, len(chr) - 1), random.randint(0, len(chr) - 1)
        new_chr[x], new_chr[y] = new_chr[y], new_chr[x]
    new_fitness = calc_fitness(new_chr)
    if new_fitness < min_fitness:
        return chr
    else:
        return new_chr

<H2 style="color:black;font-size:20px;"align="right"> این تابع کمک می کند تا وقتی راه حلی پیدا کردیم که محدودیت های بازده و ریسک را ارضا می کند ولی تعداد سهم های آن کمتر از 30 تاست،
سهم های اضافی با ضرایب بسیار کوچک تولید کنیم.</H2>

In [64]:
def num_of_nonzeros(chr):
    chromosome = np.array(chr)
    not_zero_stocks_index = np.where(chromosome != 0)[0]
    return len(not_zero_stocks_index)

def change_a_little(chr, num_of_bits):
    n = 0
    new_chr = deepcopy(chr)
    for i in range(len(chr)):
        if( n < num_of_bits):
            if chr[i] == 0:
                new_chr[i] = 0.0000000001
                n += 1
    return new_chr   

<H2 style="color:black;font-size:20px;"align="right"> در هربار جمعیت را بر اساس فیتنس سورت می کنیم، و چند درصد برتر جمعیت را دو به دو با هم جفت گیری می کنیم. با احتمال هشتاد درصد بین دو کروموزوم کراس اور انجام 
می دهیم و با احتمال 20 درصد دو کروموزوم والد مستقیما به نسل بعد منتقل می شوند. با احتمال مربوط به میوتیشن، روی هر کروموزوم جدید میوتیشن انجام می شود چک می کنیم اگر کروموزوم ها محدودیت های مساله را ارضا می کنند حلقه را تمام می کنیم و در 
غیر اینصورت همین روند را برای نسل جدید پیش می گیریم. البته من در این مساله با تغییر پارامتر ها موفق نشدم در زمان کوتاه به  نتیجه برسم</H2>

In [69]:
start_time = time.time()
population = create_population()
Pm, Pc = 0.1, 0.1
answer = []
end = False
recreat = 0

while(True):
    population.sort(key = calc_fitness)
    print("*first element after sort: ",calc_final_fitness(population[-1]))
    print(recreat)
    for i in range(int(len(population)/5)):
        o =  -1 * (2*i+2)
        e =  -1 * (2*i+1)
        if random.random() > Pc:
            population[e], population[o] = crossover(population[e], population[o])
        if random.random() > Pm:
            population[e] = mutate(population[e])
        if random.random() > Pm:
            population[o] = mutate(population[o])

        ans1 = calc_final_fitness(population[e])
        ans2 = calc_final_fitness(population[o])
        
        if  ans1 >= MIN_FITNESS :
            print(ans1)
            nz = num_of_nonzeros(population[e])
            if nz < 30:
                population[e] = change_a_little(population[e], 30 - nz)
                print("after change a little: ", calc_final_fitness(population[o]))
            answer = [population[e], ans2]
            end = 1

        elif ans2 >= MIN_FITNESS :
            print(ans2)
            nz = num_of_nonzeros(population[o])
            if nz < 30:
                population[o] = change_a_little(population[o], 30 - nz)
                print("after change a little: ", calc_final_fitness(population[o]))
            answer = [population[o], ans2]
            end = 1

    if(end):
        break

print("Answer", answer[0])
print("--- %s seconds ---" % (time.time() - start_time))

*first element after sort:  0.9792378014487673
0
*first element after sort:  1.2482687559296737
0
*first element after sort:  2.144324177411388
0
*first element after sort:  1.6840770592856342
0
*first element after sort:  1.7982843299814741
0
*first element after sort:  1.8986964328821538
0
*first element after sort:  2.421340405614857
0
*first element after sort:  2.431202258779904
0
*first element after sort:  1.9043757013678129
0
*first element after sort:  1.9043757013678129
0
*first element after sort:  2.0686488971198655
0
*first element after sort:  2.0972309515851624
0
*first element after sort:  2.0972309515851624
0
*first element after sort:  2.0972309515851624
0
*first element after sort:  2.2665565170372055
0
*first element after sort:  1.9997264252855096
0
*first element after sort:  1.965358898747025
0
*first element after sort:  1.9945793367406859
0
*first element after sort:  2.2491299956719297
0
*first element after sort:  2.2428916188854027
0
*first element after sor

*first element after sort:  4.008250568127323
0
*first element after sort:  4.085512392034589
0
*first element after sort:  4.085512392034589
0
*first element after sort:  4.329539499199087
0
*first element after sort:  4.471487370818782
0
*first element after sort:  4.951122325523393
0
*first element after sort:  5.248399197166974
0
*first element after sort:  4.430127679416816
0
*first element after sort:  5.083814840325614
0
*first element after sort:  4.7520154162680575
0
*first element after sort:  4.892518077941329
0
*first element after sort:  4.627816165675806
0
*first element after sort:  5.56883428464433
0
*first element after sort:  4.961245440508857
0
*first element after sort:  5.20058114387106
0
*first element after sort:  4.961300377022305
0
*first element after sort:  5.523931402440744
0
*first element after sort:  4.873445740310757
0
*first element after sort:  4.873445740310759
0
*first element after sort:  4.873445740310757
0
*first element after sort:  4.87344574031

*first element after sort:  5.973689676257842
0
*first element after sort:  5.973689676257842
0
*first element after sort:  5.98535071226445
0
*first element after sort:  5.730706295830085
0
*first element after sort:  5.97810434927862
0
*first element after sort:  6.163151702491315
0
*first element after sort:  6.150125696819888
0
*first element after sort:  5.8989637124199215
0
*first element after sort:  5.775284719560192
0
*first element after sort:  5.775284719560192
0
*first element after sort:  5.616743465758662
0
*first element after sort:  5.877341153631534
0
*first element after sort:  5.779978226345855
0
*first element after sort:  6.013734998999394
0
*first element after sort:  5.7600480912324565
0
*first element after sort:  5.624843439638039
0
*first element after sort:  6.3683075387570405
0
*first element after sort:  6.324428611286706
0
*first element after sort:  5.960234967138737
0
*first element after sort:  5.960234967138737
0
*first element after sort:  6.402538583

*first element after sort:  6.439465563289775
0
*first element after sort:  6.428328341397365
0
*first element after sort:  6.428328341397365
0
*first element after sort:  6.636192845866393
0
*first element after sort:  6.3084139823849705
0
*first element after sort:  6.620483845084082
0
*first element after sort:  6.129240479115924
0
*first element after sort:  6.442386803318055
0
*first element after sort:  6.130831788779763
0
*first element after sort:  6.293328558004475
0
*first element after sort:  6.293328558004477
0
*first element after sort:  6.439528892089573
0
*first element after sort:  6.26576335606691
0
*first element after sort:  6.36177636486029
0
*first element after sort:  6.3714720222310754
0
*first element after sort:  6.278553038418212
0
*first element after sort:  6.393046349489269
0
*first element after sort:  6.200479007827117
0
*first element after sort:  6.056759514109141
0
*first element after sort:  6.164167608034528
0
*first element after sort:  6.5196126537

*first element after sort:  6.952147705954314
0
*first element after sort:  7.094130009768508
0
*first element after sort:  7.094130009768508
0
*first element after sort:  6.915947231007045
0
*first element after sort:  6.915947231007045
0
*first element after sort:  7.2916256813223566
0
*first element after sort:  7.001791899592704
0
*first element after sort:  7.3194253108333065
0
*first element after sort:  7.3194253108333065
0
*first element after sort:  7.031867135727634
0
*first element after sort:  7.307189229396259
0
*first element after sort:  7.05140885264648
0
*first element after sort:  7.043932508790765
0
*first element after sort:  7.008091438063666
0
*first element after sort:  7.008091438063666
0
*first element after sort:  7.008091438063666
0
*first element after sort:  6.89059912761987
0
*first element after sort:  6.742651775988108
0
*first element after sort:  6.940286844741207
0
*first element after sort:  7.157642205944333
0
*first element after sort:  7.157642205

...

*first element after sort:  8.271260509208973
0
*first element after sort:  8.430691072485189
0
*first element after sort:  8.322173153277742
0
*first element after sort:  8.30720394772898
0
*first element after sort:  8.313022867189067
0
*first element after sort:  8.313022867189067
0
*first element after sort:  8.391392349413158
0
*first element after sort:  8.251051636761096
0
*first element after sort:  8.546099002272419
0
*first element after sort:  8.428717804535408
0
*first element after sort:  8.378638477257198
0
*first element after sort:  8.378638477257198
0
*first element after sort:  8.349175652872818
0
*first element after sort:  8.426063305990516
0
*first element after sort:  8.499034057682096
0
*first element after sort:  8.363320858279948
0
*first element after sort:  8.484972579281912
0
*first element after sort:  8.475696165742256
0
*first element after sort:  8.259213132003195
0
*first element after sort:  8.547054106569432
0
*first element after sort:  8.34591841769

*first element after sort:  8.833308429209831
0
*first element after sort:  8.833308429209831
0
*first element after sort:  8.619455394910542
0
*first element after sort:  8.92933462814389
0
*first element after sort:  8.719083802255655
0
*first element after sort:  8.831955237077523
0
*first element after sort:  8.810738685738992
0
*first element after sort:  8.646779122678801
0
*first element after sort:  8.592201601276084
0
*first element after sort:  8.537043945202221
0
*first element after sort:  8.622288134230661
0
*first element after sort:  8.601438709190353
0
*first element after sort:  8.772681548919456
0
*first element after sort:  8.63144956143397
0
*first element after sort:  8.98166311587412
0
*first element after sort:  8.63388441996149
0
*first element after sort:  8.621369861036728
0
*first element after sort:  8.62707876223894
0
*first element after sort:  8.879461007846572
0
*first element after sort:  8.662048145795254
0
*first element after sort:  8.860557971457
0


*first element after sort:  8.839249151670126
0
*first element after sort:  8.553750565429576
0
*first element after sort:  8.79062741421036
0
*first element after sort:  8.776252968665194
0
*first element after sort:  8.795280035336255
0
*first element after sort:  8.961405271602777
0
*first element after sort:  8.471806586890578
0
*first element after sort:  8.7380057921774
0
*first element after sort:  8.756831390815593
0
*first element after sort:  8.794611630059899
0
*first element after sort:  8.807338720075927
0
*first element after sort:  8.740156452787186
0
*first element after sort:  8.784890948611999
0
*first element after sort:  8.586809805393163
0
*first element after sort:  8.787459564531009
0
*first element after sort:  8.74369618254039
0
*first element after sort:  8.769210304146476
0
*first element after sort:  8.666483208653569
0
*first element after sort:  8.801089543431546
0
*first element after sort:  8.715656944148282
0
*first element after sort:  8.75853685434913

*first element after sort:  8.868631634465896
0
*first element after sort:  8.740246192897228
0
*first element after sort:  8.869338634641394
0
*first element after sort:  8.739204651305844
0
*first element after sort:  8.73258327782906
0
*first element after sort:  8.73515423539427
0
*first element after sort:  8.93187043123462
0
*first element after sort:  8.67286303494424
0
*first element after sort:  8.886668386459686
0
*first element after sort:  8.803033320498992
0
*first element after sort:  8.894097843602525
0
*first element after sort:  8.894097843602525
0
*first element after sort:  8.743906790700029
0
*first element after sort:  8.77733832126697
0
*first element after sort:  8.746137357097725
0
*first element after sort:  8.86736438977739
0
*first element after sort:  8.731342873472864
0
*first element after sort:  8.87906328072574
0
*first element after sort:  8.735834925235828
0
*first element after sort:  8.874236839120014
0
*first element after sort:  8.874236839120014
0

*first element after sort:  8.694778356084985
0
*first element after sort:  8.86605341326399
0
*first element after sort:  8.900763608720471
0
*first element after sort:  8.866401571592098
0
*first element after sort:  8.904281401349136
0
*first element after sort:  8.904281401349136
0
*first element after sort:  8.918140726374945
0
*first element after sort:  8.681831349590654
0
*first element after sort:  8.664319647529199
0
*first element after sort:  8.925760071270576
0
*first element after sort:  8.78844609933144
0
*first element after sort:  8.727327927421545
0
*first element after sort:  9.233329137474383
0
*first element after sort:  9.091179111984596
0
*first element after sort:  8.88131690567295
0
*first element after sort:  9.289558563418813
0
*first element after sort:  9.25448974300792
0
*first element after sort:  8.889436709597783
0
*first element after sort:  9.151630875628305
0
*first element after sort:  9.211083781117608
0
*first element after sort:  9.21108378111760

<H3 style="color:green;font-size:20px;"align="right">نتیجه گیری کلی</H3>
<H3 style="color:black;font-size:20px;"align="right">همان طور که دیدیم ، الگوریتم ژنتیک یکی از راه هایی است که در اکثر مواقع با سرعت بالاتر و منطقی راحت تر از الگوریتم های سرچ، ما را به جواب مساله مورد نظرمان می رساند
مهم ترین نکته این است که بتوانیم مساله را به مساله ژنتیک مدل کنیم، مفاهیم کروموزوم و ژن را استخراج کرده و با استفاده از هایپرپارامتر های مناسب به زمان و شرایط ایده آل برای گرفتن پاسخ برسیم</H3>

<H1 style="color:red;font-size:30px;"align="right">سوالات </H1>
<H3 style="color:blue;font-size:20px;"align="right">مشکلات جمعیت اولیه بسیار کم یا بسیار زیاد چیست؟ </H3>
<H3 style="color:black;font-size:20px;"align="right">جمعیت بسیار زیاد زمان زیادی برای عملیات جفت گیری نیاز دارد و جمعیت کم ممکن است شامل تمام کروموزوم های شکل دهنده به جواب بهینه نباشد و جمعیت نتواند کروموزوم ایده آل را بسازد. و با میوتیشن هم زمان زیادی طول بکشد تا کروموزوم ایده ال ساخته شود.  </H3>
<H3 style="color:blue;font-size:20px;"align="right">اگر تعداد جمعیت هر دوره زیاد شود چه تاثیری در دقت و سرعت الگوریتم می گذارد؟ </H3>
<H3 style="color:black;font-size:20px;"align="right">افزایش جمعیت در هر مرحله می تواند برای ما مشکل حافظه به وجود بیاورد و زمان  محاسبه را هم افزایش می دهد. ولی دقت را می تواند زیاد کند </H3>
<H3 style="color:blue;font-size:20px;"align="right">تاثیر عملیات های میوتیشن و کراس اور را مقایسه کنید. ایا می توان فقط از یکی استفاده کرد؟ </H3>
<H3 style="color:black;font-size:20px;"align="right">کراس اور هوش مندانه و بین کروموزوم ها با فیتنس مناسب می تواند منجر به تولید نسل هایی با فیتنس بالاتر شود اما ممکن است پس از مدتی دیگر کراس اور نسل بهتری نسازد و والد های مناسب نتوانند با هم جفت شوند. که در این صورت میوتیشن به کمک ما می آید و با تغییر ناکهانی یک ژن باعث می شود کروموزوم از مینیمم یا ماکسیمم محلی خارج شود. کراس اور نمی تواند جای میوتیشن را بگیرد. میوتیشن هم عملا بدون کراس اور و تلفیق و ژن های برتر دو والد فایده ای ندارد ولی با صرف هزینه خیلی بیشتر شاید بتواند به تنهایی با ازمون و خطای زیاد درنهایت کروموزوم را بسازد. </H3>
<H3 style="color:blue;font-size:20px;"align="right">در این مساله خاص چه راهکارهایی برای زودتر به جواب رسیدن وجود دارد؟ </H3>
<H3 style="color:black;font-size:20px;"align="right">مثلا در این مساله می دانیم که اعمال میوتیشن روی دو ضریب که صفر هستند تاثیری در فیتنس نخواهد گذاشت پس این موارد را می توانیم نادیده بگیریم و با حذف هزینه محاسبه اضافه از زمان اجرا بکاهیم </H3>
<H3 style="color:blue;font-size:20px;"align="right">چه دلایلی باعث می شود بعضی اوقات کروموزوم ها دیگر تغییر نکنند و چه راه حلی را پیشنهاد می کنید؟ </H3>
<H3 style="color:black;font-size:20px;"align="right">ممکن است چمعیت اولیه به درستی انتخاب نشده باشد و تنوع کروموزوم ها پس از مدتی کم شود و ثابت باقی بماند. در این مساله چک می شود که اگر 100 بار کروموزوم جدید با فیتنس نزدیک به هم مشاهده کرد، جمعیت را دوباره بسازد و از نو شروع به جفت گیری و تکامل کند. </H3>
<H3 style="color:blue;font-size:20px;"align="right">راه هایی برای اتمام مساله در صورتی که مساله پاسخی نداشته باشد ارایه دهید؟ </H3>
<H3 style="color:black;font-size:20px;"align="right">می توانیم محدودیت زمانی بگذاریم و یا تعیین کنیم که تا عمق معینی از نسل های جدید پیش برود و بعد از آن از حلقه خارج شود </H3>


<H3 style="color:green;font-size:20px;"align="right">منابع</H3>
<H3 style="color:black;font-size:15px;"align="right">https://github.com/Sara-Rezaeimanesh/Sudoku-Solver</H3>
<H3 style="color:black;font-size:15px;"align="right">https://github.com/nargesi-gholami/Artificial-Intelligence-projects/tree/main/AI_CA3_Genetic</H3>
<H3 style="color:black;font-size:15px;"align="right">اسلاید های درس</H3>